<a href="https://colab.research.google.com/github/Rlackdals981010/ai_Study/blob/master/hyperparams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#검증 세트
#테스트 세트를 이용하지 않고 모델이 과대적합인지 과소 적합인지 판단하는 법
#훈련 세트에서 검증 세트 분리
#훈련 : 검증 : 테스트 = 6 : 2 : 2
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
wine.head()
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
#훈련과 테스트
train_input,test_input,train_target,test_target = train_test_split(data,target, test_size=0.2,random_state=42)
#훈련과 검증
sub_input,val_input,sub_target,val_target = train_test_split(train_input,train_target,test_size=0.2,random_state=42)

print(sub_input.shape,val_input.shape)

from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target)) #과대적합

(4157, 3) (1040, 3)
0.9971133028626413
0.864423076923077


In [4]:
#교차 검증
#많은 데이터를 훈련에 사용할수록 모델이 좋음.
#하지만 검증을 너무 조금 때면 검증 점수가 불안정함 -> 교차 검증으로 안정적인 점수를 얻으면서 훈련에 데이터를 더 사용 가능
#교차검증 : 검증 세트를 떼어 내어 내어 평가하는 과정을 여러번 반복
#ex) 3-폴드 교차 검증 : 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행. 통칭 k-폴드 교차 검증이라고 한다.
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target) #교차 검증 함수. 평가할 모델 객체를 첫 번째 매개변수, 그다음 훈련 세트를 전체 전달.
print(scores) # 처음 2개의 키는 각 모델을 훈련하는 시간과 검증 시간. 각 키마다 5개의 값 -> 5-폴드 교차 검증

import numpy as np
print(np.mean(scores['test_score'])) # 교차 검증의 최종 점수


#cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. train_test_split이 선행된다면 섞을 필요가 없지만, 훈련 세트를 섞으려면 splitter(분할기)를 지정해야한다.
#cross_validate()는 회귀 모델 - KFold 분할기 사용, 분류 - stratifiedKFold사용(타깃 클래스를 골고루 나누기 위해서).

#위 코드는 이거랑 동일한것임.
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

#훈련 세트를 섞고 10-폴드 교차 검증코드는 다음과 같음
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt,train_input, train_target,cv=splitter)
print(np.mean(scores['test_score']))

{'fit_time': array([0.01003647, 0.00938535, 0.0110271 , 0.01099777, 0.01045465]), 'score_time': array([0.00160456, 0.00176096, 0.00165415, 0.00171566, 0.00197577]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487
0.855300214703487
0.8574181117533719


In [5]:
#하이퍼 파라미터 튜닝
#먼저 라이브러리 제공 기본 값을 이용 -> 검증 세트의 점수나 교차 검증을 통해 매개변수를 바꿈, 모델마다 적게 1~2개에서 많게 5~6개의 매개변수 제공.
#매개변수가 A, B가 있을때 최적의 A를 찾았다고 고정시키고 B를 찾으면 안된다. 최적의 조합을 찾아야 하기 때문
#for문으로 구현할 수 있지만 사이킷런의 그리드 서치를 사용하면 된다. cross_validate()함수도 미사용
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} #탐색할 값의 리스트
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1) #탐색 대상 모델과 params변수를 통해 그리드 서치 객체 생성
gs.fit(train_input,train_target)#훈련처럼 호출하여 그리드 서치 객체가 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5회 실시
dt = gs.best_estimator_
print(dt.score(train_input,train_target))
#그리드 서치로 찾은 최적의 매개변수
print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [6]:
#각 매개변수에서 수행한 교차검증의 평균 점수
print(gs.cv_results_['mean_test_score'])
#argmax() 함수를 통해 가장 큰 값의 인덱스 추출
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


1. 탐색할 매개변수 지정
2. 훈련 세트에서 그리드 서치 수행 -> 최상의 평균 점수가 나오는 매개변수 조합을 찾음 print(gs.best_params_)
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련세트 아님!) 전체 훈련 세트를 사용해 최종 모델 훈련. 이 모델도 그리드 서치 객체에 저장

In [7]:
#더 복잡한 매개변수 조합 탐색
#min_impurity_decrease <- 노드 분할을 위한 불순도 감소 최소량 지정
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001), #np.arange()는 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 떄까지 세 번째 매개변수를 계속 더한 배열을 만든다. 0.0001~0.0009로 9개 원소
          'max_depth' : range(5,20,1), #range np.arange랑 비슷한데 정수만 가능함. max_depth를 5에서 20까지 1씩 증가하면서 15개의 값을 만듦 15개의 원소
          'min_samples_split' : range(2,100,10) # 2부터 100까지 10씩증가 10개의 원소
          }#총 9*15*10 = 1350개의 원소
          #기본 5-폴드 교차 검증을 수행하므로 모델은 6750개

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [8]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731


In [12]:
#랜덤 서치
#매개변수 값이 수치일 때, 값의 범위나 간격을 미리 정하기 어렵다.
#또한, 조건이 많아 그리드 서치 수행 시간이 오래 걸릴 경우, 랜덤 서치 사용
#매개변수 값의 목록이 아닌 샘플링할 수 있는 확률 분포 객체를 전달한다.
from scipy.stats import uniform, randint # 2개의 확률 분포 클래스
#주어진 범위에서 고르게 값을 뽑느다. -> 균등 분포에서 샘플링
#randint -> 정수값, uniform -> 실숫값.
rgen = randint(0,10) # 0에서 10 사이의 범위를 갖는 randint 객체
rgen.rvs(10) # 10개 샘플링 -> 고르게 되는거같지가 않음

array([0, 4, 2, 2, 7, 7, 4, 0, 9, 6])

In [13]:
np.unique(rgen.rvs(1000), return_counts=True) #1000개 샘플링, 어느정도 고르게 추출된거로 보임

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105,  85, 111,  96,  97,  85,  99, 110,  95, 117]))

In [16]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.4792551 , 0.00502965, 0.67734991, 0.08599626, 0.78537964,
       0.7393569 , 0.06931625, 0.23557291, 0.6048615 , 0.88769804])

In [25]:
params = {'min_impurity_decrease' : uniform(0.0001,0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25),
          }

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,
                        n_iter=100, n_jobs=-1, random_state=42) #params에 정의된 매개변수 범위에서 총 100번을 샘플링하여 교차 검증을 수행하고, 최적의 매개변수 조합을 찾음
gs.fit(train_input,train_target)

print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt=gs.best_estimator_
print(dt.score(test_input,test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
